In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('cooling_durations_full_output.csv')

In [35]:
df.head()

,time_stamp,moisture_active1,moisture_active2,oxygen,lid,co2,device_id,temperature_active1,temperature_active2,temperature_active3,temperature_active4,temperature_curing1,temperature_curing2,moisture_curing1,moisture_curing2,max_temp_compartment_A,max_temp_compartment_B,cooling_duration_A,cooling_duration_B
0,2024-01-01,53.414959,54.445332,14.585502,1,5.155804,dev1,30.745071,29.792604,30.971533,32.284545,NaN,NaN,NaN,NaN,30.745071,32.284545,NaN,NaN
1,2024-01-02,54.431544,55.309214,14.804655,1,5.425923,dev1,33.870992,33.871017,36.591041,35.373374,NaN,NaN,NaN,NaN,33.871017,36.591041,NaN,NaN
2,2024-01-03,55.332008,55.207526,15.543138,1,4.899575,dev1,37.740233,39.258285,37.749318,37.745850,NaN,NaN,NaN,NaN,39.258285,37.749318,NaN,NaN
3,2024-01-04,55.714561,56.200347,15.559422,1,5.018819,dev1,43.029610,39.796746,40.079290,41.823235,NaN,NaN,NaN,NaN,43.029610,41.823235,NaN,NaN
4,2024-01-05,51.243475,51.566163,15.328532,1,5.071450,dev1,45.369642,47.360260,45.526853,44.770433,NaN,NaN,NaN,NaN,47.360260,45.526853,NaN,NaN


Creating an actionable column composting phase which can be used to identify and classify composting phases for future data.

The classify_phase function is based on the research..however if the client could provide more information about the compartment A and compartment B if there is a particular temperature threshold that allows the compost to move from compartment A to compartment will solidify accurately classifying 

In [36]:
def classify_phase(df):
    max_temp_A = df['max_temp_compartment_A']
    max_temp_B = df['max_temp_compartment_B']
    oxygen = df['oxygen']
    temp_curing1 = df['temperature_curing1']
    temp_curing2 = df['temperature_curing2']
    max_active = np.maximum(max_temp_A, max_temp_B)

    max_curing = np.maximum(temp_curing1, temp_curing2)
    active_empty_rows = max_active.isna()

    conditions = [
        active_empty_rows & max_curing.notna() & (max_curing >= 20) & (max_curing <= 40),
        active_empty_rows & max_curing.notna() & (max_curing < 20),
        ~active_empty_rows & max_active.notna() & (max_active > 50) ,
        ~active_empty_rows & max_active.notna() & (max_active > 20) & (max_active < 50)
    ]

    results = [
        'Curing',
        'Matured',
        'Thermophilic',
        'Mesophilic'
    ]

    df['composting_phase'] = np.select(conditions, results, default='Mesophilic')

    return df


In [37]:
phase_df = classify_phase(df)

In [38]:
phase_df.head()

,time_stamp,moisture_active1,moisture_active2,oxygen,lid,co2,device_id,temperature_active1,temperature_active2,temperature_active3,temperature_active4,temperature_curing1,temperature_curing2,moisture_curing1,moisture_curing2,max_temp_compartment_A,max_temp_compartment_B,cooling_duration_A,cooling_duration_B,composting_phase
0,2024-01-01,53.414959,54.445332,14.585502,1,5.155804,dev1,30.745071,29.792604,30.971533,32.284545,NaN,NaN,NaN,NaN,30.745071,32.284545,NaN,NaN,Mesophilic
1,2024-01-02,54.431544,55.309214,14.804655,1,5.425923,dev1,33.870992,33.871017,36.591041,35.373374,NaN,NaN,NaN,NaN,33.871017,36.591041,NaN,NaN,Mesophilic
2,2024-01-03,55.332008,55.207526,15.543138,1,4.899575,dev1,37.740233,39.258285,37.749318,37.745850,NaN,NaN,NaN,NaN,39.258285,37.749318,NaN,NaN,Mesophilic
3,2024-01-04,55.714561,56.200347,15.559422,1,5.018819,dev1,43.029610,39.796746,40.079290,41.823235,NaN,NaN,NaN,NaN,43.029610,41.823235,NaN,NaN,Mesophilic
4,2024-01-05,51.243475,51.566163,15.328532,1,5.071450,dev1,45.369642,47.360260,45.526853,44.770433,NaN,NaN,NaN,NaN,47.360260,45.526853,NaN,NaN,Mesophilic


The composting phase can be used as the target value for successfully identify and classify composting phases when the sensor data are received

In [ ]:
# Data CSV file
df.reset_index().to_csv("composting_phase_output.csv", index=False)